# Машинное обучение, ФКН ВШЭ

## Практическое задание 2. Exploratory Data Analysis и линейная регрессия

### Общая информация
Дата выдачи: 12.10.2024

Мягкий дедлайн: 23:59MSK 27.10.2024

Жесткий дедлайн: 23:59MSK 31.10.2024

### О задании
В этом задании мы попытаемся научиться анализировать данные и выделять из них полезные признаки. Мы также научимся пользоваться `sklearn`, а заодно привыкнем к основным понятиям машинного обучения.

### Оценивание и штрафы
Каждая из задач имеет определенную «стоимость» (указана в скобках около задачи). Максимально допустимая оценка за работу — 10 баллов. Проверяющий имеет право снизить оценку за неэффективную реализацию или неопрятные графики.

**Обратите внимание**, что в каждом разделе домашнего задания есть оцениваниемые задачи и есть вопросы. Вопросы дополняют задачи и направлены на то, чтобы проинтерпретировать или обосновать происходящее. Код без интерпретации не имеет смысла, поэтому отвечать на вопросы обязательно — за отсутствие ответов мы будем снижать баллы за задачи. Если вы ответите на вопросы, но не напишете корректный код к соответствующим оцениваемым задачам, то баллы за такое выставлены не будут.

Задание выполняется самостоятельно. «Похожие» решения считаются плагиатом и все задействованные студенты (в том числе те, у кого списали) не могут получить за него больше 0 баллов (подробнее о плагиате см. на странице курса). Если вы нашли решение какого-то из заданий (или его часть) в открытом источнике, необходимо указать ссылку на этот источник в отдельном блоке в конце вашей работы (скорее всего вы будете не единственным, кто это нашел, поэтому чтобы исключить подозрение в плагиате, необходима ссылка на источник).

### Формат сдачи
Задания сдаются через систему Anytask. Инвайт можно найти на странице курса. Присылать необходимо ноутбук с выполненным заданием. Сам ноутбук называйте в формате homework-practice-02-linregr-Username.ipynb, где Username — ваша фамилия.

Для удобства проверки самостоятельно посчитайте свою максимальную оценку (исходя из набора решенных задач) и укажите ниже.

Оценка: xx.

В этом ноутбуке используется библиотека `folium` для визуализации карт. Она работает в google colab!

In [ ]:
!pip install folium

In [ ]:
import folium

m = folium.Map(location=(50.97178, 9.79418), zoom_start=15)
m

Если вы всё сделали правильно, то выше должна открыться карта

### 📌 **Памятка по дз2**

Обрабатывать данные можно любым способом (`polars`, `pandas`, `pyspark`, ...), который вам нравится и запускается в ноутбуке, и любой библиотекой для визуализации (`matplotlib`, `seaborn`, `plotly`, ...). Пользуйтесь на здоровье, но!

> Учтите, что `polars` всё ещё молодой и не все библиотеки его поддерживают. `sklearn` во многом уже работает, но на всякий случай не стесняйтесь пользоваться методами `pl.DataFrame.to_pandas()` или `pl.DataFrame.to_numpy()`    

> Впрочем, `pandas` тоже не лишён проблем. `sklearn` принимает `pd.DataFrame` почти везде, проблемы очень редки, но если вдруг возникнет какая-то беда с шейпами, или ещё какой-нибудь казус, а вы уверены, что всё правильно, пользуйтесь `pd.DataFrame.to_numpy()`. Это не всегда решает проблему, но часто помогает понять, что на самом деле не так. При желании можно передавать и `Iterable`, и sparse-матрицы

> Графики должны быть понятные, читаемые и пр. Консультируйтесь с памяткой из дз1 в задании на график

> Для воспроизводимости результатов не забывайте пользоваться `np.random.seed(...)`, при необходимости чистите мусор `gc.collect()`, лучше в каждой ячейке

> Все результаты должны быть получены в ноутбуке. На каждый **Вопрос** долен быть дан **Ответ** (письменно (в Markdown например) или кодом)

In [1]:
import datetime

import numpy as np
import polars as pl
import plotly.express as px

In [3]:
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.compose import ColumnTransformer
from sklearn.linear_model import Lasso, LinearRegression, Ridge
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import GridSearchCV, ShuffleSplit, train_test_split
from sklearn.pipeline import Pipeline, FeatureUnion # prefer ColumnTransformer over FeatureUnion since it allows to take a subset of columns
from sklearn.preprocessing import OneHotEncoder, StandardScaler

In [ ]:
# This allows one to not use exessively .set_output(transform='polars') and also provides ~40% performance boost vs default
import sklearn
sklearn.set_config(transform_output = 'polars')

## Часть 0. Подготовка (1 балл)

**Задание 1 (1 балл)**. Мы будем работать с данными из соревнования [New York City Taxi Trip Duration](https://www.kaggle.com/c/nyc-taxi-trip-duration/overview), в котором нужно было предсказать длительность поездки на такси. Скачайте обучающую выборку из этого соревнования и загрузите ее:

In [ ]:
!pip install kaggle

In [5]:
!mkdir ~/.kaggle
!ln -s /data/ml-course-hse/config/.kaggle/kaggle.json ~/.kaggle/kaggle.json
!chmod 600 ~/.kaggle/kaggle.json

In [ ]:
!mkdir -p /data/ml-course-hse/2024-fall/homework-practice-02-linregr && kaggle competitions download -c nyc-taxi-trip-duration -p /data/ml-course-hse/2024-fall/homework-practice-02-linregr

In [ ]:
!unzip /data/ml-course-hse/2024-fall/homework-practice-02-linregr/nyc-taxi-trip-duration.zip -d /data/ml-course-hse/2024-fall/homework-practice-02-linregr/
!unzip /data/ml-course-hse/2024-fall/homework-practice-02-linregr/train.zip                  -d /data/ml-course-hse/2024-fall/homework-practice-02-linregr/
!unzip /data/ml-course-hse/2024-fall/homework-practice-02-linregr/test.zip                   -d /data/ml-course-hse/2024-fall/homework-practice-02-linregr/
!unzip /data/ml-course-hse/2024-fall/homework-practice-02-linregr/sample_submission.zip      -d /data/ml-course-hse/2024-fall/homework-practice-02-linregr/

Обратите внимание на колонки `pickup_datetime` и `dropoff_datetime`. `dropoff_datetime` был добавлена организаторами только в обучающую выборку, то есть использовать эту колонку нельзя, давайте удалим ее. В `pickup_datetime` записаны дата и время начала поездки. Чтобы с ней было удобно работать, давайте преобразуем даты в `datetime`-объекты

In [ ]:
# читаем датафрейм
df = pl.scan_csv(
    "../../../../data/ml-course-hse/2024-fall/homework-practice-02-linregr/train.csv"
)
df_test = pl.scan_csv(
    "../../../../data/ml-course-hse/2024-fall/homework-practice-02-linregr/test.csv"
)
print(
    f"Number of rows in train: {df.collect().shape[0]}; in test: {df_test.collect().shape[0]} rows"
)
print(df.collect().estimated_size("mb"), "mb")

In [5]:
df = df.select(pl.exclude("dropoff_datetime")).with_columns(
    pl.col("pickup_datetime").str.to_datetime(
        format="%Y-%m-%d %H:%M:%S", time_zone="EST"
    )
)
df_test = df_test.with_columns(
    pl.col("pickup_datetime").str.to_datetime(
        format="%Y-%m-%d %H:%M:%S", time_zone="EST"
    )
)

In [ ]:
df.describe()

In [ ]:
df.head().collect()

В колонке `trip_duration` записано целевое значение, которое мы хотим предсказывать. Давайте посмотрим на распределение таргета в обучающей выборке. Для этого нарисуйте его гистограмму:

In [8]:
layout_dict = dict(
    margin=dict(l=20, r=20, t=40, b=20),
    width=600,
    height=400,
    paper_bgcolor="LightSteelBlue",
    title_font_size=14,
    xaxis_title_font_size=12,
    yaxis_title_font_size=12,
)

## reference:
# fig.update_layout(yaxis_title="trip count", yaxis_title_font_size=12)
# fig.update_layout(yaxis=dict(title=dict(text="trip count", font_size=12)))

In [ ]:
fig = px.histogram(
    df.collect(),
    x="trip_duration",
    title="Count of trip durations in test dataset",
    nbins=100,
)
fig.update_layout(**layout_dict)

**Вопрос**: Что можно сказать о целевой переменной по гистограмме её значений?

В соревновании в качестве метрики качества использовалось RMSLE:
$$\text{RMSLE}(X, y, a) = \sqrt{\frac{1}{\ell}\sum_{i=1}^{\ell} \big(\log{(y_i + 1)} - \log{(a(x_i) + 1)}\big)^2}$$

**Вопрос**: Как вы думаете, почему авторы соревнования выбрали именно RMSLE, а не RMSE?

На семинаре мы рассматривали несколько моделей линейной регрессии в `sklearn`, но каждая из них оптимизировала среднеквадратичную ошибку (MSE), а не RMSLE. Давайте проделаем следующий трюк: будем предсказывать не целевую переменную, а ее *логарифм*. Обозначим $\hat{y}_i = \log{(y_i + 1)}$ — модифицированный таргет, а $\hat{a}(x_i)$ — предсказание модели, которая обучалась на $\hat{y}_i$, то есть логарифм таргета. Чтобы предсказать исходное значение, мы можем просто взять экспоненту от нашего предсказания: $a(x_i) = \exp(\hat{a}(x_i)) - 1$.

**Вопрос**: Покажите, что оптимизация RMSLE для модели $a$ эквивалентна оптимизации MSE для модели $\hat{a}$.

**Доказательство**: ╰( ͡° ͜ʖ ͡° )つ──☆*:・ﾟ

Итак, мы смогли свести задачу оптимизации RMSLE к задаче оптимизации MSE, которую мы умеем решать! Кроме того, у логарифмирования таргета есть еще одно полезное свойство. Чтобы его увидеть, добавьте к нашей выборке колонку `log_trip_duration` (воспользуйтесь `np.log1p`) и нарисуйте гистограмму модифицированного таргета по обучающей выборке. Удалите колонку со старым таргетом.

In [9]:
df = df.with_columns(pl.col("trip_duration").log1p().alias("log_trip_duration"))
df_test = df_test.with_columns(
    pl.col("trip_duration").log1p().alias("log_trip_duration")
)

Чтобы иметь некоторую точку отсчета, давайте посчитаем значение метрики при наилучшем константном предсказании:

In [ ]:
def rmsle(log1p_y_true, log1p_y_pred):
    return (log1p_y_true - log1p_y_pred).pow(2).mean() ** 0.5


rmsle_best_const = rmsle(
    df.select("log_trip_duration").collect()["log_trip_duration"],
    df.select(pl.col("log_trip_duration").mean()).collect().item(),
)
print(rmsle_best_const)
assert np.allclose(rmsle_best_const, 0.79575, 1e-4)

## Часть 1. Изучаем `pickup_datetime` (2 балла)

**Задание 2 (0.25 баллов)**. Для начала давайте посмотрим, сколько всего было поездок в каждый из дней. Постройте график зависимости количества поездок от дня в году:

In [ ]:
fig = px.bar(
    df.group_by(pl.col("pickup_datetime").dt.date().alias("date"))
    .agg(pl.len().alias("trip_count"))
    .collect(),
    x="date",
    y="trip_count",
    title="Trip count by date",
)
# fig.update_layout(yaxis_title="trip count", yaxis_title_font_size=12)
fig.update_layout(yaxis=dict(title=dict(text="trip count")))
fig.update_layout(**layout_dict)

In [ ]:
df.group_by(pl.col("pickup_datetime").dt.date().alias("date")).agg(
    pl.len().alias("trip_count")
).sort("trip_count").head(10).collect()

**Вопрос**: Вы, вероятно, заметили, что на графике есть 2 периода с аномально маленькими количествами поездок. Вычислите, в какие даты происходили эти скачки вниз и найдите информацию о том, что происходило в эти дни в Нью-Йорке.

Нарисуйте графики зависимости количества поездок от дня недели и от часов в сутках:

In [ ]:
fig = px.bar(
    df.group_by(pl.col("pickup_datetime").dt.weekday().alias("weekday"))
    .agg(pl.len().alias("trip_count"))
    .sort("weekday")
    .with_columns(
        pl.col("weekday")
        .cast(pl.String)
        .replace(
            {
                "1": "Mon",
                "2": "Tue",
                "3": "Wed",
                "4": "Thu",
                "5": "Fri",
                "6": "Sat",
                "7": "Sun",
            }
        ),
    )
    .collect(),
    x="weekday",
    y="trip_count",
    title="Trip count by weekday",
)
fig.update_layout(yaxis=dict(title=dict(text="trip count", font_size=12)))
fig.update_layout(**layout_dict)
fig.show()


fig = px.bar(
    df.group_by(pl.col("pickup_datetime").dt.hour().alias("hour"))
    .agg(pl.len().alias("trip_count"))
    .sort("hour")
    .with_columns(pl.col("hour").cast(pl.String))
    .collect(),
    x="hour",
    y="trip_count",
    title="Trip count by hour",
)
fig.update_layout(yaxis=dict(title=dict(text="trip count", font_size=12)))
fig.update_layout(**layout_dict)
fig.show()

**Задание 3 (0.5 баллов)**. Нарисуйте на одном графике зависимости количества поездок от часа в сутках для разных месяцев (разные кривые, соответствующие разным месяцам, окрашивайте в разные цвета, поищите, как это делается). Аналогично нарисуйте зависимости количества поездок от часа в сутках для разных дней недели.

In [ ]:
fig = px.line(
    df.group_by(
        pl.col("pickup_datetime").dt.hour().alias("hour"),
        pl.col("pickup_datetime").dt.month().alias("month"),
    )
    .agg(pl.len().alias("trip_count"))
    .sort(["month", "hour"])
    .with_columns(
        pl.col("hour").cast(pl.String),
        pl.col("month")
        .cast(pl.String)
        .replace(
            {"1": "Jan", "2": "Feb", "3": "Mar", "4": "Apr", "5": "May", "6": "Jun"}
        ),
    )
    .collect(),
    x="hour",
    y="trip_count",
    color="month",
    title="Trip count by hour and month",
)

fig.update_layout(**layout_dict)
fig.show()

fig = px.line(
    df.group_by(
        pl.col("pickup_datetime").dt.hour().alias("hour"),
        pl.col("pickup_datetime").dt.weekday().alias("weekday"),
    )
    .agg(pl.len().alias("trip_count"))
    .sort(["weekday", "hour"])
    .with_columns(
        pl.col("hour").cast(pl.String),
        pl.col("weekday")
        .cast(pl.String)
        .replace(
            {
                "1": "Mon",
                "2": "Tue",
                "3": "Wed",
                "4": "Thu",
                "5": "Fri",
                "6": "Sat",
                "7": "Sun",
            }
        ),
    )
    .collect(),
    x="hour",
    y="trip_count",
    color="weekday",
    title="Trip count by hour and weekday",
)

fig.update_layout(**layout_dict)
fig.show()

**Вопрос**: Какие выводы можно сделать, основываясь на графиках выше? Выделяются ли какие-нибудь дни недели? Месяца? Время суток? С чем это связано?

**Задание 4 (0.5 баллов)**. Разбейте выборку на обучающую и тестовую в отношении 7:3 (используйте `train_test_split` из `sklearn`). По обучающей выборке нарисуйте график зависимости среднего логарифма времени поездки от дня недели. Затем сделайте то же самое, но для часа в сутках и дня в году.

In [11]:
xtrain, xval = train_test_split(df.collect(), test_size=0.3, random_state=120)
xtrain = xtrain.lazy()
xval = xval.lazy()

In [ ]:
fig = px.bar(
    xtrain.group_by(pl.col("pickup_datetime").dt.weekday().alias("weekday"))
    .agg(pl.col("log_trip_duration").mean())
    .sort("weekday")
    .with_columns(
        pl.col("weekday")
        .cast(pl.String)
        .replace(
            {
                "1": "Mon",
                "2": "Tue",
                "3": "Wed",
                "4": "Thu",
                "5": "Fri",
                "6": "Sat",
                "7": "Sun",
            }
        )
    )
    .collect(),
    x="weekday",
    y="log_trip_duration",
    title="Avg Log trip duration by weekday",
)
fig.update_layout(yaxis=dict(title=dict(text="log trip duration", font_size=12)))
fig.update_layout(**layout_dict)
fig.show()

fig = px.bar(
    xtrain.group_by(pl.col("pickup_datetime").dt.hour().alias("hour"))
    .agg(pl.col("log_trip_duration").mean())
    .sort("hour")
    .with_columns(
        pl.col("hour").cast(pl.String),
    )
    .collect(),
    x="hour",
    y="log_trip_duration",
    title="Avg Log trip duration by hour",
)
fig.update_layout(yaxis=dict(title=dict(text="log trip duration", font_size=12)))
fig.update_layout(**layout_dict)
fig.show()

fig = px.bar(
    xtrain.group_by(pl.col("pickup_datetime").dt.date().alias("date"))
    .agg(pl.col("log_trip_duration").mean())
    .sort("date")
    .collect(),
    x="date",
    y="log_trip_duration",
    title="Avg Log trip duration by date",
)
fig.update_layout(yaxis=dict(title=dict(text="log trip duration", font_size=12)))
fig.update_layout(**layout_dict)
fig.show()

**Вопрос**: Похожи ли графики зависимости таргета от дня недели и от часа в сутках на аналогичные графики для количества поездок? Почему? Что происходит со средним таргетом в те два аномальных периода, что мы видели выше? Почему так происходит? Наблюдаете ли вы какой-нибудь тренд на графике зависимости `log_trip_duration` от номера дня в году?

Добавьте следующие признаки на основе `pickup_datetime`:
1. День недели
2. Месяц
3. Час
4. Является ли период аномальным (два бинарных признака, соответствующие двум аномальным периодам)
5. Номер дня в году

In [12]:
xtrain = xtrain.with_columns(pl.col("pickup_datetime").dt.weekday().alias("weekday"))
xtrain = xtrain.with_columns(
    pl.col("pickup_datetime").dt.month().cast(pl.String).alias("month")
)
xtrain = xtrain.with_columns(pl.col("pickup_datetime").dt.hour().alias("hour"))
xtrain = xtrain.with_columns(
    pl.col("pickup_datetime").dt.date().cast(pl.Int64).alias("date")
)
xtrain = xtrain.with_columns(
    pl.col("pickup_datetime")
    .dt.date()
    .is_between(datetime.date(2016, 1, 23), datetime.date(2016, 1, 29))
    .alias("is_blizzard")
)
xtrain = xtrain.with_columns(
    (pl.col("pickup_datetime").dt.date() == datetime.date(2016, 5, 30)).alias(
        "is_memorial"
    )
)

xval = xval.with_columns(pl.col("pickup_datetime").dt.weekday().alias("weekday"))
xval = xval.with_columns(
    pl.col("pickup_datetime").dt.month().cast(pl.String).alias("month")
)
xval = xval.with_columns(pl.col("pickup_datetime").dt.hour().alias("hour"))
xval = xval.with_columns(
    pl.col("pickup_datetime").dt.date().cast(pl.Int64).alias("date")
)
xval = xval.with_columns(
    pl.col("pickup_datetime")
    .dt.date()
    .is_between(datetime.date(2016, 1, 23), datetime.date(2016, 1, 29))
    .alias("is_blizzard")
)
xval = xval.with_columns(
    (pl.col("pickup_datetime").dt.date() == datetime.date(2016, 5, 30)).alias(
        "is_memorial"
    )
)

Итак, мы уже создали некоторое количество признаков.

**Вопрос**: Какие из признаков _стоит рассматривать в этой задаче_   как категориальные, а какие - как численные? Почему?

**Задание 5 (0.75 баллов)**. Обучите `Ridge`-регрессию с параметрами по умолчанию, закодировав все категориальные признаки с помощью `OneHotEncoder`. Численные признаки отмасштабируйте с помощью `StandardScaler`. Используйте только признаки, которые мы выделили в этой части задания.

In [13]:
categorical = ["hour", "weekday", "month", "is_memorial", "is_blizzard"]
numeric_features = [
    "passenger_count",
    "pickup_latitude",
    "pickup_longitude",
    "dropoff_longitude",
    "dropoff_latitude",
]

column_transformer = ColumnTransformer(
    [
        ("ohe", OneHotEncoder(handle_unknown="ignore", drop="if_binary"), categorical),
        ("scaling", StandardScaler(), numeric_features),
    ]
)

pipeline = Pipeline(
    steps=[("ohe_and_scaling", column_transformer), ("regression", Ridge())]
)

model0 = pipeline.fit(
    xtrain.collect(), xtrain.select("log_trip_duration").collect()["log_trip_duration"]
)  # restricting before collecting would work much faster

In [ ]:
ytrain_pred = model0.predict(xtrain.collect())
yval_pred = model0.predict(xval.collect())

print(
    f"""
    Constant prediction RMSLE {rmsle_best_const:.3f}
    Train RMSLE: {mean_squared_error(xtrain.select('log_trip_duration').collect()['log_trip_duration'], ytrain_pred)**.5:.3f}
    Validation RMSLE: {mean_squared_error(xval.select('log_trip_duration').collect()['log_trip_duration'], yval_pred)**.5:.3f}
"""
)

In [ ]:
np.exp(
    xtrain.select("log_trip_duration").mean().collect().item()
    + np.array([0.796, 0.773]).reshape(-1, 1) * np.linspace(-2, 2, 5)
)

In [ ]:
pl.DataFrame(
    {
        "variable": model0["ohe_and_scaling"]["scaling"].feature_names_in_,
        "coefficient": model0["regression"].coef_[-len(numeric_features) :],
    }
)

## Часть 2. Изучаем координаты (3 балла)
Мы уже очень хорошо изучили данные о времени начала поездки, давайте теперь посмотрим на информацию о координатах начала и конца поездки. Мы подготовили для вас функцию, которая на карте рисует точки начала или конца поездки. Примеры ее вызова вы найдете ниже. Обратите внимание, что в эту функцию мы передаем лишь небольшой кусочек данных, посколько иначе функция будет работать очень долго

In [ ]:
from typing import Iterable


def show_circles_on_map(
    latitude_values: Iterable[float],
    longitude_values: Iterable[float],
    color: str = "blue",
) -> folium.Map:
    """
    The function draws map with circles on it.
    The center of the map is the mean of coordinates passed in data.
    Works best on samples of size < 10k, too costly otherwise

    latitude_values: sample latitude values of a dataframe
    longitude_column: sample longitude values of a dataframe
    color: the color of circles to be drawn
    """

    location = (np.mean(latitude_values), np.mean(longitude_values))
    m = folium.Map(location=location)

    for lat, lon in zip(latitude_values, longitude_values):
        folium.Circle(
            radius=100, location=(lat, lon), color=color, fill_color=color, fill=True
        ).add_to(m)

    return m

Напишите функция, которая вернет значения колонок `pickup_latitude`, `pickup_longitude` на каком-нибудь разумном кусочке датафрейма, например, `df.sample(1000)` и покажите на карте, используя `show_circles_on_map`

In [ ]:
# ╰( ͡° ͜ʖ ͡° )つ──☆*:・ﾟ
# your_lat_values = ...
# your_lon_values = ...

# show_circles_on_map(your_lat_values, your_lon_values, color="midnightblue")

**Вопрос**: Какие пункты (или скопления точек, в количестве 2-3), по вашему мнению, выделяются на карте от основной массы и могут быть полезны для нашей задачи? Почему вы их выбрали? В чём особенность этих скоплений точек для нашей задачи?

**Задание 6 (0.75 балл)**. Как мы все прекрасно помним, $t = s / v_{\text{ср}}$, поэтому очевидно, что самым сильным признаком будет расстояние, которое необходимо проехать. Мы не можем посчитать точное расстояние, которое необходимо преодолеть такси, но мы можем его оценить, посчитав кратчайшее расстояние между точками начала и конца поездки. Чтобы корректно посчитать расстояние между двумя точками на Земле, можно использовать функцию `haversine`. Также можно воспользоваться кодом с первого семинара. Посчитайте кратчайшее расстояние для объектов и запишите его в колонку `haversine`:

In [16]:
def haversine(lat1, lat2, lon1, lon2):
    R = 3958.8
    _lat1, _lat2, _lon1, _lon2 = (x * np.pi / 180 for x in (lat1, lat2, lon1, lon2))
    return (
        2
        * R
        * np.arcsin(
            np.sqrt(
                np.square(np.sin(0.5 * (_lon1 - _lon2)))
                + np.cos(_lon1)
                * np.cos(_lon2)
                * np.square(np.sin(0.5 * (_lat1 - _lat2)))
            )
        )
    )

In [17]:
R = 3958.8
xtrain = (
    xtrain.with_columns(
        pl.col(
            "pickup_latitude",
            "pickup_longitude",
            "dropoff_latitude",
            "dropoff_longitude",
        )
        .mul(np.pi / 180)
        .name.suffix("_")
    )
    .with_columns(
        pl.col("pickup_longitude_")
        .sub("dropoff_longitude_")
        .mul(0.5)
        .sin()
        .pow(2)
        .add(
            pl.col("pickup_latitude_")
            .sub("dropoff_latitude_")
            .mul(0.5)
            .sin()
            .pow(2)
            .mul(pl.col("pickup_longitude_").cos())
            .mul(pl.col("dropoff_longitude_").cos())
            .alias("lat_term")
        )
        .sqrt()
        .arcsin()
        .mul(2 * R)
        .alias("haversine")
    )
    .select(
        pl.exclude(
            "pickup_latitude_",
            "pickup_longitude_",
            "dropoff_latitude_",
            "dropoff_longitude_",
        )
    )
)

xval = (
    xval.with_columns(
        pl.col(
            "pickup_latitude",
            "pickup_longitude",
            "dropoff_latitude",
            "dropoff_longitude",
        )
        .mul(np.pi / 180)
        .name.suffix("_")
    )
    .with_columns(
        pl.col("pickup_longitude_")
        .sub("dropoff_longitude_")
        .mul(0.5)
        .sin()
        .pow(2)
        .add(
            pl.col("pickup_latitude_")
            .sub("dropoff_latitude_")
            .mul(0.5)
            .sin()
            .pow(2)
            .mul(pl.col("pickup_longitude_").cos())
            .mul(pl.col("dropoff_longitude_").cos())
            .alias("lat_term")
        )
        .sqrt()
        .arcsin()
        .mul(2 * R)
        .alias("haversine")
    )
    .select(
        pl.exclude(
            "pickup_latitude_",
            "pickup_longitude_",
            "dropoff_latitude_",
            "dropoff_longitude_",
        )
    )
)

Так как мы предсказываем логарифм времени поездки и хотим, чтобы наши признаки были линейно зависимы с этой целевой переменной, нам нужно логарифмировать расстояние: $\log t = \log s - \log{v_{\text{ср}}}$. Запишите логарифм `haversine` в отдельную колонку `log_haversine`:

In [18]:
xtrain = xtrain.with_columns(pl.col("haversine").add(1e-3).log().alias("log_haversine"))
xval = xval.with_columns(pl.col("haversine").add(1e-3).log().alias("log_haversine"))

Посчитайте корреляцию и убедитесь, что логарифм расстояния лучше отражает таргет, чем просто расстояние:

In [ ]:
print(
    f"Correlation before log {xtrain.select(pl.corr('haversine', 'log_trip_duration')).collect().item():.3f}"
)
print(
    f"Correlation after log {xtrain.select(pl.corr('log_haversine', 'log_trip_duration')).collect().item():.3f}"
)

**Задание 7 (0.75 балла)**. Давайте изучим среднюю скорость движения такси. Посчитайте среднюю скорость для каждого объекта обучающей выборки, разделив `haversine` на `trip_duration`, и нарисуйте гистограмму ее распределения

In [ ]:
fig = px.histogram(
    xtrain.with_columns(
        pl.col("haversine").truediv("trip_duration").mul(3600).alias("speed")
    ).collect(),
    x="speed",
    title="speed distribution",
    nbins=100,
)
fig.update_layout(**layout_dict)

Как можно видеть по гистограмме, для некоторых объектов у нас получились очень больше значения скоростей. Нарисуйте гистограмму по объектам, для которых значение скорости получилось разумным (например, можно не включать рассмотрение объекты, где скорость больше некоторой квантили):

In [ ]:
fig = px.histogram(
    xtrain.with_columns(
        pl.col("haversine").truediv("trip_duration").mul(3600).alias("speed")
    )
    .filter(pl.col("speed") <= pl.col("speed").quantile(0.975))
    .collect(),
    x="speed",
    title="speed distribution",
    nbins=100,
)
fig.update_layout(**layout_dict)

Для каждой пары (день недели, час суток) посчитайте медиану скоростей. Нарисуйте Heatmap-график, где по осям будут дни недели и часы, а в качестве значения функции - медиана скорости

In [ ]:
fig = px.imshow(
    xtrain.with_columns(
        pl.col("haversine").truediv("trip_duration").mul(3600).alias("speed")
    )
    .sort("hour", "weekday")
    .with_columns(
        pl.col("weekday")
        .cast(pl.String)
        .replace(
            {
                "1": "Mon",
                "2": "Tue",
                "3": "Wed",
                "4": "Thu",
                "5": "Fri",
                "6": "Sat",
                "7": "Sun",
            }
        )
    )
    .collect()
    .pivot("weekday", index="hour", values="speed", aggregate_function="median")
    .select(pl.exclude("hour")),
    y=list(range(24)),
    labels=dict(x="Day of week", y="hour"),
    title="Speed by day of week and hour",
)
# fig.update_xaxes(side="top")
fig.update_layout(layout_dict)

Не забудьте удалить колонку со значением скорости из данных!

**Вопрос**: Почему значение скорости нельзя использовать во время обучения?

In [ ]:
# it contains information on target variable

**Вопрос**: Посмотрите внимательно на график и скажите, в какие моменты времени скорость минимальна; максимальна.

Создайте признаки "поездка совершается в период пробок" и "поездка совершается в период свободных дорог" (естественно, они не должен зависеть от скорости!):

In [23]:
xtrain = xtrain.with_columns(
    (pl.col("weekday").is_between(1, 5) & pl.col("hour").is_between(8, 18)).alias(
        "is_rush"
    ),
    (
        (pl.col("weekday").is_between(1, 5) & pl.col("hour").is_between(0, 5)).or_(
            pl.col("weekday").is_between(6, 7) & pl.col("hour").is_between(4, 7)
        )
    ).alias("is_free"),
)

xval = xval.with_columns(
    (pl.col("weekday").is_between(1, 5) & pl.col("hour").is_between(8, 18)).alias(
        "is_rush"
    ),
    (
        (pl.col("weekday").is_between(1, 5) & pl.col("hour").is_between(0, 5)).or_(
            pl.col("weekday").is_between(6, 7) & pl.col("hour").is_between(4, 7)
        )
    ).alias("is_free"),
)

**Задание 8 (0.25 балла)**. Для каждого из замеченных вами выше 2-3 пунктов добавьте в выборку по два признака:
- началась ли поездка в этом пункте
- закончилась ли поездка в этом пункте

Как вы думаете, почему эти признаки могут быть полезны?

In [24]:
xtrain = xtrain.with_columns(
    pl.col("pickup_latitude")
    .is_between(40.7685745910316, 40.78092423293102)
    .and_(pl.col("pickup_longitude").is_between(-73.88553158521124, -73.85549084411662))
    .alias("lga_pickup"),
    pl.col("dropoff_latitude")
    .is_between(40.7685745910316, 40.78092423293102)
    .and_(
        pl.col("dropoff_longitude").is_between(-73.88553158521124, -73.85549084411662)
    )
    .alias("lga_dropoff"),
    pl.col("pickup_latitude")
    .is_between(40.64014671430768, 40.649348476637805)
    .and_(
        pl.col("pickup_longitude").is_between(-73.796050739231967, -73.77550641673797)
    )
    .alias("jfk_pickup"),
    pl.col("dropoff_latitude")
    .is_between(40.64014671430768, 40.649348476637805)
    .and_(
        pl.col("dropoff_longitude").is_between(-73.796050739231967, -73.77550641673797)
    )
    .alias("jfk_dropoff"),
)

xval = xval.with_columns(
    pl.col("pickup_latitude")
    .is_between(40.7685745910316, 40.78092423293102)
    .and_(pl.col("pickup_longitude").is_between(-73.88553158521124, -73.85549084411662))
    .alias("lga_pickup"),
    pl.col("dropoff_latitude")
    .is_between(40.7685745910316, 40.78092423293102)
    .and_(
        pl.col("dropoff_longitude").is_between(-73.88553158521124, -73.85549084411662)
    )
    .alias("lga_dropoff"),
    pl.col("pickup_latitude")
    .is_between(40.64014671430768, 40.649348476637805)
    .and_(
        pl.col("pickup_longitude").is_between(-73.796050739231967, -73.77550641673797)
    )
    .alias("jfk_pickup"),
    pl.col("dropoff_latitude")
    .is_between(40.64014671430768, 40.649348476637805)
    .and_(
        pl.col("dropoff_longitude").is_between(-73.796050739231967, -73.77550641673797)
    )
    .alias("jfk_dropoff"),
)

Для каждого из созданных признаков нарисуйте "ящик с усами" (aka boxplot) распределения логарифма времени поездки

In [ ]:
fig = px.box(
    xtrain.select(pl.col("log_trip_duration", "is_rush", "is_free")).collect(),
    y="log_trip_duration",
    x="is_rush",
    color="is_free",
)
fig.update_layout(layout_dict)
fig.show()

In [ ]:
fig = px.box(
    xtrain.select(pl.col("log_trip_duration", "jfk_pickup", "jfk_dropoff")).collect(),
    y="log_trip_duration",
    x="jfk_pickup",
    color="jfk_dropoff",
)
fig.update_layout(layout_dict)
fig.show()

In [ ]:
fig = px.box(
    xtrain.select(pl.col("log_trip_duration", "lga_pickup", "lga_dropoff")).collect(),
    y="log_trip_duration",
    x="lga_pickup",
    color="lga_dropoff",
)
fig.update_layout(layout_dict)
fig.show()

**Вопрос**: судя по графикам, как вы думаете, хорошими ли получились эти признаки?

<img src="https://www.dropbox.com/s/xson9nukz5hba7c/map.png?raw=1" align="right" width="20%" style="margin-left: 20px; margin-bottom: 20px">

**Задание 9 (1 балл)**. Сейчас мы почти что не используем сами значения координат. На это есть несколько причин: по отдельности рассматривать широту и долготу не имеет особого смысла, стоит рассматривать их вместе. Во-вторых, понятно, что зависимость между нашим таргетом и координатами не линейная. Чтобы как-то использовать координаты, можно прибегнуть к следующему трюку: обрамим область с наибольшим количеством поездок прямоугольником (как на рисунке). Разобьем этот прямоугольник на ячейки. Каждой точке сопоставим номер ее ячейки, а тем точкам, что не попали ни в одну из ячеек, сопоставим значение -1.

Напишите трансформер, который сначала разбивает показанную на рисунке область на ячейки, а затем создает два признака: номер ячейки, в которой началась поездка, и номер ячейки, в которой закончилась поездка. Количество строк и столбцов выберите самостоятельно.

Обратите внимание, что все вычисления должны быть векторизованными, трансформер не должен модифицировать передаваемую ему выборку inplace, а все необходимые статистики (если они вдруг нужны) нужно считать только по обучающей выборке в методе `fit`:

In [25]:

# paradigm
# ss = StandardScaler()
# ss.set_output(transform='polars')
# ss.fit(xtrain.select(pl.col('dropoff_longitude', 'pickup_latitude')).collect())
# ss.transform(xval.select(pl.col('dropoff_longitude', 'pickup_latitude')).collect())

# TransformerMixin implements fit_transform for you,
# applying your fit and transform consistently


class MapGridTransformer(BaseEstimator, TransformerMixin):
    def __init__(
        self, xname, yname, nrow=1, ncol=1, rotate_degrees=0, min_count=None, tol=0.01
    ):
        """
        xname: name of x coordinate feature
        yname: name of y coordinate feature
        nrow: number of rows for produced feature
        ncol: number of columns for produced feature
        rotate_degrees: rotate the map by that many degrees prior to producing the grid for feature
        min_count: rectangles within the grid with less than this amount of observations will be labelled (-1, -1), by default controlled by tol*fitting sample size
        tol: quantile margin to determine boundaries of the grid
        """
        self.xname = xname
        self.yname = yname
        self.nrow = nrow
        self.ncol = ncol
        self.rotate_degrees = rotate_degrees
        self.tol = tol
        self.min_count = min_count

    def show_map(self):
        # you may want to visualize cells
        pass

    def rotate(self, X):
        cos = pl.lit(self.rotate_degrees).radians().cos()
        sin = pl.lit(self.rotate_degrees).radians().sin()
        return X.select(
            xnorm=pl.col(self.xname).mul(cos) + pl.col(self.yname).mul(sin),
            ynorm=pl.col(self.xname).mul(sin).neg() + pl.col(self.yname).mul(cos),
        )

    def fit(self, X=None, y=None):
        if self.min_count is None:
            self.min_count = (
                X.select(pl.len()).item() * self.tol / (self.nrow * self.ncol)
            )

        cuts = self.rotate(X).select(
            xcuts=pl.concat_list(
                pl.col("xnorm").quantile(q)
                for q in np.linspace(self.tol / 2, 1 - self.tol / 2, 1 + self.ncol)
            ),
            ycuts=pl.concat_list(
                pl.col("ynorm").quantile(q)
                for q in np.linspace(self.tol / 2, 1 - self.tol / 2, 1 + self.nrow)
            ),
        )
        self.xcuts = cuts["xcuts"].item()
        self.ycuts = cuts["ycuts"].item()
        return self

    def transform(self, X, y=None):
        return (
            self.rotate(X)
            .select(
                xi=pl.col("xnorm")
                .cut(self.xcuts, labels=[str(x) for x in range(-1, self.ncol + 1)])
                .cast(pl.Int64()),
                yi=pl.col("ynorm")
                .cut(self.ycuts, labels=[str(x) for x in range(-1, self.nrow + 1)])
                .cast(pl.Int64()),
            )
            .with_columns(
                pl.col("xi").replace(self.ncol, -1),
                pl.col("yi").replace(self.nrow, -1),
                pl.len().over("xi", "yi").alias("count"),
            )
            .select(
                pl.when(
                    (pl.col("count") < self.min_count)
                    | (pl.col("xi") == -1)
                    | (pl.col("yi") == -1)
                )
                .then(-1)
                .otherwise(pl.col("xi").add(pl.col("yi").mul(self.ncol)))
                .alias("grid_point")
            )
        )

    def get_feature_names_out(
        self, names
    ):  # this is so that get_feature_names_out method works for pipelines using this transformer
        return ["grid_point"]

In [ ]:
mgt = MapGridTransformer('pickup_longitude', 'pickup_latitude', nrow=8, ncol=8, rotate_degrees=-29, min_count=400, tol=0.01)
mgt.fit(xtrain.collect())

In [ ]:
from IPython.display import display

with pl.Config(set_tbl_rows=10):
    display(mgt.transform(xval.collect()).group_by('grid_point').agg(pl.len()).sort('len'))

**Задание 10 (0.25 балла)**. Обучите `Ridge`-регрессию со стандартными параметрами на признаках, которые мы выделили к текущему моменту. Категориальные признаки закодируйте через one-hot-кодирование, числовые признаки отмасштабируйте.

In [140]:
enc = OneHotEncoder(
    drop=[-1, -1], sparse_output=False
)  # -1 is the category assigned to the points outside of the grid
enc.set_output(transform="polars")

pickup_dropoff = Pipeline(
    [
        (
            "pickup_dropoff_grid",
            ColumnTransformer(
                [
                    (
                        "mgt_pickup",
                        MapGridTransformer(
                            "pickup_longitude",
                            "pickup_latitude",
                            nrow=6,
                            ncol=6,
                            rotate_degrees=-29,
                            min_count=200,
                            tol=0.01,
                        ),  # .set_output(transform='polars')
                        ["pickup_longitude", "pickup_latitude"],
                    ),
                    (
                        "mgt_dropoff",
                        MapGridTransformer(
                            "dropoff_longitude",
                            "dropoff_latitude",
                            nrow=6,
                            ncol=6,
                            rotate_degrees=-29,
                            min_count=200,
                            tol=0.01,
                        ),  # .set_output(transform='polars')
                        ["dropoff_longitude", "dropoff_latitude"],
                    ),
                ]
            ),  # .set_output(transform='polars')
        ),
        ("enc_pickup_dropoff", enc),
    ]
)  # .set_output(transform='polars')

categorical = [
    "hour",
    "weekday",
    "month",
    "is_memorial",
    "is_blizzard",
    "is_rush",
    "is_free",
    "jfk_pickup",
    "jfk_dropoff",
    "lga_pickup",
    "lga_dropoff",
]
numeric_features = ["log_haversine"]

features = ColumnTransformer(
    [
        (
            "scaling",
            StandardScaler(),  # .set_output(transform='polars')
            numeric_features,
        ),
        (
            "ohe_all",
            OneHotEncoder(
                handle_unknown="ignore", drop="if_binary", sparse_output=False
            ),  # .set_output(transform='polars')
            categorical,
        ),
    ]
)  # .set_output(transform='polars')

pipeline = Pipeline(
    [
        (
            "features",
            FeatureUnion(
                [("pickup_dropoff", pickup_dropoff), ("features", features)]
            ),  # .set_output(transform='polars')
        ),  # .set_output(transform='polars')
        ("regression", Ridge()),
    ]
)

In [141]:
model1 = pipeline.fit(xtrain.collect(), xtrain.select("log_trip_duration").collect()["log_trip_duration"])

In [105]:
# Side note -- most of the time is taken not by training but by feature evaluation
# pipeline['features'].transform(xtrain.collect())

In [ ]:
ytrain_pred = model1.predict(xtrain.collect())
yval_pred = model1.predict(xval.collect())

print(
    f"""
    Constant prediction RMSLE {rmsle_best_const:.3f}
    Train RMSLE: {mean_squared_error(xtrain.select('log_trip_duration').collect()['log_trip_duration'], ytrain_pred)**.5:.3f}
    Validation RMSLE: {mean_squared_error(xval.select('log_trip_duration').collect()['log_trip_duration'], yval_pred)**.5:.3f}
"""
)

In [ ]:
np.exp(
    xtrain.select("log_trip_duration").mean().collect().item()
    + np.array([0.796, 0.546]).reshape(-1, 1) * np.linspace(-2, 2, 5)
)

In [ ]:
# # this is useful for eyeballing resulting coefficients
# with pl.Config(set_tbl_rows=150, set_fmt_str_lengths=60):
#     display(
#         pl.DataFrame(
#             {
#                 "variable": model1["features"].get_feature_names_out(),
#                 "coefficient": model1["regression"].coef_,
#             }
#         )
#     )

In [149]:
# prestoring features makes it much faster:
pipeline_no_regressor = Pipeline(
    [
        (
            "features",
            FeatureUnion(
                [("pickup_dropoff", pickup_dropoff), ("features", features)]
            ),
        ),
        # ("regression", Ridge()),
    ]
)

xtrain_trsf = pipeline_no_regressor.fit_transform(xtrain.collect()).lazy()
xval_trsf = pipeline_no_regressor.fit_transform(xval.collect()).lazy()

In [ ]:
model_trsf = Ridge()
model_trsf.fit(xtrain_trsf.collect(), xtrain.select("log_trip_duration").collect()["log_trip_duration"])

In [ ]:
ytrain_pred = model_trsf.predict(xtrain_trsf.collect())
yval_pred = model_trsf.predict(xval_trsf.collect())

print(
    f"""
    Constant prediction RMSLE {rmsle_best_const:.3f}
    Train RMSLE: {mean_squared_error(xtrain.select('log_trip_duration').collect()['log_trip_duration'], ytrain_pred)**.5:.3f}
    Validation RMSLE: {mean_squared_error(xval.select('log_trip_duration').collect()['log_trip_duration'], yval_pred)**.5:.3f}
"""
)

## Часть 3. Изучаем оставшиеся признаки (1 балл)

**Задание 11 (0.75 баллов)**. У нас осталось еще 3 признака, которые мы не исследовали: `vendor_id`, `passenger_count` и `store_and_fwd_flag`.

**Вопрос**: Подумайте, почему каждый из этих признаков может быть потенциально полезным.

Посчитайте, сколько есть уникальных значений у каждого из этих признаков:

In [ ]:
with pl.Config(set_tbl_rows=10):
    display(df.group_by("passenger_count").agg(pl.len().alias("len")).sort("len", descending=True).collect())
    display(df.group_by("vendor_id").agg(pl.len().alias("len")).sort("len", descending=True).collect())
    display(df.group_by("store_and_fwd_flag").agg(pl.len().alias("len")).sort("len", descending=True).collect())

Постройте "ящики с усами" распределений логарифма времени поездки в зависимости от значений каждого из признаков

In [ ]:
# passengers >=3 is a feature
fig = px.box(
    xtrain.select(pl.col("log_trip_duration", "passenger_count")).collect(),
    y="log_trip_duration",
    x="passenger_count",
)
fig.update_layout(layout_dict)
fig.show()

In [ ]:
# vendor_id doesn't look like a good feature
fig = px.box(
    xtrain.select(pl.col("log_trip_duration", "vendor_id")).collect(),
    y="log_trip_duration",
    x="vendor_id",
)
fig.update_layout(layout_dict)
fig.show()

In [ ]:
# store_and_fwd_flag is a feature
fig = px.box(
    xtrain.select(pl.col("log_trip_duration", "store_and_fwd_flag")).collect(),
    y="log_trip_duration",
    x="store_and_fwd_flag",
)
fig.update_layout(layout_dict)
fig.show()

Переведите признаки `vendor_id` и `store_and_fwd_flag` в значения $\{0;1\}$

In [155]:
xtrain = xtrain.with_columns(
    store_and_fwd_flag=pl.col("store_and_fwd_flag") == "Y",
    is_many_passengers=pl.col("passenger_count") >= 3,
)
xval = xval.with_columns(
    store_and_fwd_flag=pl.col("store_and_fwd_flag") == "Y",
    is_many_passengers=pl.col("passenger_count") >= 3,
)

**Вопрос**: Основываясь на графиках выше, как вы думаете, будут ли эти признаки сильными?

**Задание 12 (0.25 баллов)**. Проверьте свои предположения, обучив модель в том числе и на этих трех признаках. Обучайте `Ridge`-регрессию со стандартными параметрами. Категориальные признаки закодируйте one-hot-кодированием, а численные отмасштабируйте.

In [308]:
from sklearn.pipeline import make_pipeline

In [412]:
pickup_dropoff = Pipeline(
    [
        (
            "pickup_dropoff_grid",
            ColumnTransformer(
                [
                    (
                        "mgt_pickup",
                        MapGridTransformer(
                            "pickup_longitude",
                            "pickup_latitude",
                            nrow=6,
                            ncol=6,
                            rotate_degrees=-29,
                            min_count=200,
                            tol=0.01,
                        ),
                        ["pickup_longitude", "pickup_latitude"],
                    ),
                    (
                        "mgt_dropoff",
                        MapGridTransformer(
                            "dropoff_longitude",
                            "dropoff_latitude",
                            nrow=6,
                            ncol=6,
                            rotate_degrees=-29,
                            min_count=200,
                            tol=0.01,
                        ),
                        ["dropoff_longitude", "dropoff_latitude"],
                    ),
                ]
            ),
        ),
        (
            "enc_pickup_dropoff",
            OneHotEncoder(drop=[-1, -1], sparse_output=False)
        ),
    ]
)

categorical = [
    "hour",
    "weekday",
    "month",
    "is_memorial",
    "is_blizzard",
    "is_rush",
    "is_free",
    "jfk_pickup",
    "jfk_dropoff",
    "lga_pickup",
    "lga_dropoff",
    "is_many_passengers",
    "store_and_fwd_flag",
]
numeric_features = ["log_haversine"]

features = ColumnTransformer(
    [
        ("scaling", StandardScaler(), numeric_features),
        (
            "ohe_all",
            OneHotEncoder(
                handle_unknown="ignore", drop="if_binary", sparse_output=False
            ),
            categorical,
        ),
    ]
)

pipeline_no_regressor = Pipeline(
    [
        (
            "features",
            FeatureUnion([("pickup_dropoff", pickup_dropoff), ("features", features)]),
        ),
        # ("regression", Ridge()),
    ]
)

In [413]:
xtrain_trsf = pipeline_no_regressor.fit_transform(xtrain.collect()).lazy()
xval_trsf = pipeline_no_regressor.fit_transform(xval.collect()).lazy()

In [ ]:
model2 = Ridge()
model2.fit(xtrain_trsf.collect(), xtrain.select("log_trip_duration").collect()["log_trip_duration"])

In [ ]:
# Side note -- most of the time is taken not by training but by feature evaluation
# pipeline['features'].transform(xtrain.collect())

In [ ]:
ytrain_pred = model2.predict(xtrain_trsf.collect())
yval_pred = model2.predict(xval_trsf.collect())

print(
    f"""
    Constant prediction RMSLE {rmsle_best_const:.3f}
    Train RMSLE: {mean_squared_error(xtrain.select('log_trip_duration').collect()['log_trip_duration'], ytrain_pred)**.5:.3f}
    Validation RMSLE: {mean_squared_error(xval.select('log_trip_duration').collect()['log_trip_duration'], yval_pred)**.5:.3f}
"""
)

In [ ]:
np.exp(
    xtrain.select("log_trip_duration").mean().collect().item()
    + np.array([0.796, 0.546]).reshape(-1, 1) * np.linspace(-2, 2, 5)
)

Если признаки не дали какого-то ощутимого улучшения метрики, их можно выбросить из данных.

## Часть 4. Улучшаем модель (3 балла)

**Задание 13 (1 балл)**. В наших данных есть нетипичные объекты (выбросы, или outliers): с аномально маленьким времени поездки, с очень большим пройденным расстоянием или очень большими остатками регрессии. В этом задании предлагается исключить такие объекты из обучающей выборки. Для этого нарисуйте гистограммы распределения упомянутых выше величин, выберите объекты, которые можно назвать выбросами, и очистите __обучающую выборку__ от них.

Отметим, что хотя эти объекты и выглядят как выбросы, в тестовой выборке тоже скорее всего будут объекты с такими же странными значениями целевой переменной и/или признаков. Поэтому, возможно, чистка обучающей выборки приведёт к ухудшению качества на тесте. Тем не менее, всё равно лучше удалять выбросы из обучения, чтобы модель получалась более разумной и интерпретируемой.

In [164]:
xtrain = xtrain.filter(
    ~((pl.col("trip_duration") == 0.0).and_(pl.col("haversine") > 0.0)),
    pl.col("haversine").truediv("trip_duration").mul(3600) <= 120,
)

In [172]:
xtrain_trsf = pipeline_no_regressor.fit_transform(xtrain.collect()).lazy()
xval_trsf = pipeline_no_regressor.fit_transform(xval.collect()).lazy()

Сейчас у нас очень много категориальных признаков. В категориальных признаках могут содержаться редкие категории, обычно это плохо: модель сильно переобучается на таких примерах. Для каждого категориального признака объедините действительно редкие категории в одну, если такие имеются (т.е. если категории действительно редкие).

In [ ]:
# ╰( ͡° ͜ʖ ͡° )つ──☆*:・ﾟ

Обучите модель на очищенных данных и посчитайте качество на тестовой выборке.

**Задание 14 (1 балл)**. После OneHot-кодирования количество признаков в нашем датасете сильно возрастает. Посчитайте колиество признаков до и после кодирования категориальных признаков.

In [ ]:
# already showed a table with coefficients

Попробуйте обучить не `Ridge`-, а `Lasso`-регрессию. Какой метод лучше?

In [ ]:
model3 = Lasso(alpha = 1e-2)
model3.fit(xtrain_trsf.collect(), xtrain.select("log_trip_duration").collect()["log_trip_duration"])

In [ ]:
ytrain_pred = model3.predict(xtrain_trsf.collect())
yval_pred = model3.predict(xval_trsf.collect())

print(
    f"""
    Constant prediction RMSLE {rmsle_best_const:.3f}
    Train RMSLE: {mean_squared_error(xtrain.select('log_trip_duration').collect()['log_trip_duration'], ytrain_pred)**.5:.3f}
    Validation RMSLE: {mean_squared_error(xval.select('log_trip_duration').collect()['log_trip_duration'], yval_pred)**.5:.3f}
"""
)

Разбейте _обучающую выборку_ на обучающую и валидационную в отношении 8:2. По валидационной выборке подберите оптимальные значения параметра регуляризации (по логарифмической сетке) для `Ridge` и `Lasso`, на тестовой выборке измерьте качество лучшей полученной модели.

In [ ]:
alphas = np.logspace(-5, 5, 18)
cv = ShuffleSplit(1, random_state=312, test_size = 0.2) # can't naturally track coefficients, so would have to do brute force loop

model_ridge = Ridge()
searcher = GridSearchCV(
    model_ridge,
    [{"alpha": alphas}], # could be "regression__alpha" if not pre-storing features and using full Pipeline object with regressor
    scoring="neg_root_mean_squared_error",
    cv=cv,
    n_jobs=-1,
)
searcher.fit(
    xtrain_trsf.collect(), xtrain.select("log_trip_duration").collect()["log_trip_duration"]
)

In [ ]:
#  to avoid confusion here: best_estimator_ is refitted with best parameter (even though CV uses 80%-20% fit)
ytrain_pred = searcher.best_estimator_.predict(xtrain_trsf.collect())
yval_pred = searcher.best_estimator_.predict(xval_trsf.collect())

print(
    f"""
    Constant prediction RMSLE {rmsle_best_const:.3f}
    Train RMSLE: {mean_squared_error(xtrain.select('log_trip_duration').collect()['log_trip_duration'], ytrain_pred)**.5:.3f}
    Validation RMSLE: {mean_squared_error(xval.select('log_trip_duration').collect()['log_trip_duration'], yval_pred)**.5:.3f}
"""
)

In [ ]:
fig = px.line(
    pl.DataFrame({'alpha':searcher.param_grid[0]['alpha'], 'score':-searcher.cv_results_["mean_test_score"]}),
    x='alpha',
    y='score'
)
fig.update_layout(yaxis=dict(title=dict(text="trip count", font_size=12)))
fig.update_layout(**layout_dict)
fig.show()

Сохраните наилучшее значение, как бенчмарк на будущее

In [ ]:
# ╰( ͡° ͜ʖ ͡° )つ──☆*:・ﾟ
best_rmsle = ...

Для каждого перебранного `alpha` для Lasso посчитайте количество нулевых весов в модели и нарисуйте график зависимости его от `alpha`. Как сильно придётся потерять в качестве, если мы хотим с помощью Lasso избавиться хотя бы от половины признаков?

In [ ]:
from sklearn.linear_model import LassoCV

alphas = np.logspace(-8, -2, 14)
lasso_cv = LassoCV(cv=cv, alphas=alphas, n_jobs=-1)
lasso_cv.fit(xtrain_trsf.collect(), xtrain.select("log_trip_duration").collect()["log_trip_duration"])

In [ ]:
path = lasso_cv.path(
    xtrain_trsf.collect(),
    xtrain.select("log_trip_duration").collect()["log_trip_duration"],
    alphas=alphas,
)
path_coeff = pl.DataFrame(dict(zip(lasso_cv.feature_names_in_, path[1]))).with_columns(
    alpha=pl.Series(path[0])
)
fig = px.line(
    path_coeff.unpivot(index="alpha", variable_name="coeff", value_name="value"),
    x="alpha",
    y="value",
    color="coeff",
)
fig.update_layout(yaxis=dict(title=dict(font_size=12)))
fig.update_layout(showlegend=False)
fig.update_traces(opacity=.4)
fig.update_layout(**layout_dict)
fig.show()

In [ ]:
fig = px.line(
    pl.DataFrame({'alpha':lasso_cv.alphas_, 'score':lasso_cv.mse_path_**.5}),
    x='alpha',
    y='score'
)
fig.update_layout(yaxis=dict(title=dict(font_size=12)))
fig.update_layout(**layout_dict)
fig.show()

In [ ]:
# path_coeff.select(pl.exclude("alpha") == 0, ).select(n_zeros=pl.sum_horizontal(pl.all()))
path_coeff.select(alpha=pl.col("alpha"), n_zeros=pl.sum_horizontal(0 == pl.exclude("alpha")))

<img src="https://www.dropbox.com/s/wp4jj0599np17lh/map_direction.png?raw=1" width="20%" align="right" style="margin-left: 20px">

**Задание 15 (1 балл)**. Часто бывает полезным использовать взаимодействия признаков (feature interactions), то есть строить новые признаки на основе уже существующих. Выше мы разбили карту Манхэттена на ячейки и придумали признаки "из какой ячейки началась поездка" и "в какой ячейке закончилась поездка".

Давайте попробуем сделать следующее: посчитаем, сколько раз встречается каждая возможная пара этих признаков в нашем датасете и выберем 100 самых частых пар. Закодируем поездки с этими частыми парами как категориальный признак, остальным объектам припишем -1. Получается, что мы закодировали, откуда и куда должно было ехать такси.

**Вопрос**: Почему такой признак потенциально полезный? Почему линейная модель не может самостоятельно "вытащить" эту информацию, ведь у нее в распоряжении есть признаки "из какой ячейки началась поездка" и "в какой ячейке закончилась поездка"?

In [404]:
class OccurrenceRankEncoder(BaseEstimator, TransformerMixin):
    def __init__(self, n_common=1):
        self.n_common = n_common

    def fit(self, X=None, y=None):
        return self

    def transform(self, X, y=None):
        return (
            X.join(
                X.group_by(pl.all()).agg(occurrence_rank=pl.len()),
                on=X.columns,
                how="inner",
            )
            .select(pl.col("occurrence_rank").rank(descending=True, method='dense'))
            .select(
                pl.when(pl.col("occurrence_rank") <= self.n_common)
                .then(pl.col("occurrence_rank"))
                .otherwise(0)
            )
        )

    def get_feature_names_out(
        self, names
    ):  # this is so that get_feature_names_out method works for pipelines using this transformer
        return ["occurrence_rank"]

In [414]:
pickup_dropoff = Pipeline(
    [
        (
            "pickup_dropoff_grid",
            ColumnTransformer(
                [
                    (
                        "mgt_pickup",
                        MapGridTransformer(
                            "pickup_longitude",
                            "pickup_latitude",
                            nrow=6,
                            ncol=6,
                            rotate_degrees=-29,
                            min_count=200,
                            tol=0.01,
                        ),
                        ["pickup_longitude", "pickup_latitude"],
                    ),
                    (
                        "mgt_dropoff",
                        MapGridTransformer(
                            "dropoff_longitude",
                            "dropoff_latitude",
                            nrow=6,
                            ncol=6,
                            rotate_degrees=-29,
                            min_count=200,
                            tol=0.01,
                        ),
                        ["dropoff_longitude", "dropoff_latitude"],
                    ),
                ]
            ),
        ),
        (
            "two_picks",
            FeatureUnion(
                [
                    # ('passthrough', make_pipeline('passthrough')), # ('passthrough', ColumnTransformer([], remainder='passthrough')),
                    (
                        "occ_ohe",
                        Pipeline(
                            [
                                ("occ_rank", OccurrenceRankEncoder(100)),
                                (
                                    "occ_enc",
                                    OneHotEncoder(drop=[0], sparse_output=False),
                                ),
                            ]
                        ),
                    ),
                    (
                        "enc_pickup_dropoff1",
                        OneHotEncoder(drop=[-1, -1], sparse_output=False),
                    ),
                ]
            ),
        ),
    ]
)

categorical = [
    "hour",
    "weekday",
    "month",
    "is_memorial",
    "is_blizzard",
    "is_rush",
    "is_free",
    "jfk_pickup",
    "jfk_dropoff",
    "lga_pickup",
    "lga_dropoff",
    "is_many_passengers",
    "store_and_fwd_flag",
]
numeric_features = ["log_haversine"]

features = ColumnTransformer(
    [
        ("scaling", StandardScaler(), numeric_features),
        (
            "ohe_all",
            OneHotEncoder(
                handle_unknown="ignore", drop="if_binary", sparse_output=False
            ),
            categorical,
        ),
    ]
)

pipeline_no_regressor = Pipeline(
    [
        (
            "features",
            FeatureUnion([("pickup_dropoff", pickup_dropoff), ("features", features)]),
        ),
        # ("regression", Ridge()),
    ]
)

# pickup_dropoff.fit_transform(xval.collect()).select(pl.sum('occ_ohe__occurrence_rank_1', 'occ_ohe__occurrence_rank_99'))

In [415]:
xtrain_trsf = pipeline_no_regressor.fit_transform(xtrain.collect()).lazy()
xval_trsf = pipeline_no_regressor.fit_transform(xval.collect()).lazy()

Заново обучите модель (`Ridge`, если она дала более высокое качество в предыдущих экспериментах, и `Lasso` иначе) на новых даннных и посчитайте качество на тестовой выборке

In [ ]:
model4 = Ridge()
model4.fit(xtrain_trsf.collect(), xtrain.select("log_trip_duration").collect()["log_trip_duration"])

In [ ]:
ytrain_pred = model4.predict(xtrain_trsf.collect())
yval_pred = model4.predict(xval_trsf.collect())

print(
    f"""
    Constant prediction RMSLE {rmsle_best_const:.3f}
    Train RMSLE: {mean_squared_error(xtrain.select('log_trip_duration').collect()['log_trip_duration'], ytrain_pred)**.5:.3f}
    Validation RMSLE: {mean_squared_error(xval.select('log_trip_duration').collect()['log_trip_duration'], yval_pred)**.5:.3f}
"""
)

In [ ]:
alphas = np.logspace(-8, -2, 14)
lasso_cv = LassoCV(cv=cv, alphas=alphas, n_jobs=-1)
lasso_cv.fit(xtrain_trsf.collect(), xtrain.select("log_trip_duration").collect()["log_trip_duration"])

In [ ]:
fig = px.line(
    pl.DataFrame({'alpha':lasso_cv.alphas_, 'score':lasso_cv.mse_path_**.5}),
    x='alpha',
    y='score'
)
fig.update_layout(yaxis=dict(title=dict(font_size=12)))
fig.update_layout(**layout_dict)
fig.show()

In [ ]:
ytrain_pred = lasso_cv.predict(xtrain_trsf.collect())
yval_pred = lasso_cv.predict(xval_trsf.collect())

print(
    f"""
    Constant prediction RMSLE {rmsle_best_const:.3f}
    Train RMSLE: {mean_squared_error(xtrain.select('log_trip_duration').collect()['log_trip_duration'], ytrain_pred)**.5:.3f}
    Validation RMSLE: {mean_squared_error(xval.select('log_trip_duration').collect()['log_trip_duration'], yval_pred)**.5:.3f}
"""
)

**Задание 16 (бонус, 1 балл)**. Где, как не для нашей задачи, считать манхэттенское расстояние?

**Вопрос**: Найдите, что такое манхэттенское расстояние и почему оно так называется. Как оно нам может помочь?

Введите систему координат на нашей карте так, чтобы оси были параллельны улицам Манхэттена, и добавьте сначала в данные признак "манхэттенское расстояние между пунктом отправления и пунктом назначения", а затем и логарифм этого признака. Посчитайте корреляцию между вашим новыми признаком и таргетом; между `log_haversine` и таргетом. В каком случае корреляция больше?

Нарисуйте карту, где покажете выбранные оси. Чтобы мы могли проверить вашу работу, просьба сделать скрин этой карты и приложить картинку (если мы откроем ваш ноутбук, виджеты отображаться не будут).

In [ ]:
rotate_degrees = -29.0
cos = pl.lit(rotate_degrees).radians().cos()
sin = pl.lit(rotate_degrees).radians().sin()
xtrain = xtrain.with_columns(
    manhattan_dist=(
        pl.col("pickup_longitude").mul(cos)
        + pl.col("pickup_latitude").mul(sin)
        - pl.col("dropoff_longitude").mul(cos)
        - pl.col("dropoff_latitude").mul(sin)
    ).abs()
    + (
        pl.col("pickup_longitude").mul(sin).neg()
        + pl.col("pickup_latitude").mul(cos)
        - pl.col("dropoff_longitude").mul(sin).neg()
        - pl.col("dropoff_latitude").mul(cos)
    ).abs()
)

xval = xval.with_columns(
    manhattan_dist=(
        pl.col("pickup_longitude").mul(cos)
        + pl.col("pickup_latitude").mul(sin)
        - pl.col("dropoff_longitude").mul(cos)
        - pl.col("dropoff_latitude").mul(sin)
    ).abs()
    + (
        pl.col("pickup_longitude").mul(sin).neg()
        + pl.col("pickup_latitude").mul(cos)
        - pl.col("dropoff_longitude").mul(sin).neg()
        - pl.col("dropoff_latitude").mul(cos)
    ).abs()
)

In [434]:
pickup_dropoff = Pipeline(
    [
        (
            "pickup_dropoff_grid",
            ColumnTransformer(
                [
                    (
                        "mgt_pickup",
                        MapGridTransformer(
                            "pickup_longitude",
                            "pickup_latitude",
                            nrow=6,
                            ncol=6,
                            rotate_degrees=-29,
                            min_count=200,
                            tol=0.01,
                        ),
                        ["pickup_longitude", "pickup_latitude"],
                    ),
                    (
                        "mgt_dropoff",
                        MapGridTransformer(
                            "dropoff_longitude",
                            "dropoff_latitude",
                            nrow=6,
                            ncol=6,
                            rotate_degrees=-29,
                            min_count=200,
                            tol=0.01,
                        ),
                        ["dropoff_longitude", "dropoff_latitude"],
                    ),
                ]
            ),
        ),
        (
            "two_picks",
            FeatureUnion(
                [
                    # ('passthrough', make_pipeline('passthrough')), # ('passthrough', ColumnTransformer([], remainder='passthrough')),
                    (
                        "occ_ohe",
                        Pipeline(
                            [
                                ("occ_rank", OccurrenceRankEncoder(100)),
                                (
                                    "occ_enc",
                                    OneHotEncoder(drop=[0], sparse_output=False),
                                ),
                            ]
                        ),
                    ),
                    (
                        "enc_pickup_dropoff1",
                        OneHotEncoder(drop=[-1, -1], sparse_output=False),
                    ),
                ]
            ),
        ),
    ]
)

categorical = [
    "hour",
    "weekday",
    "month",
    "is_memorial",
    "is_blizzard",
    "is_rush",
    "is_free",
    "jfk_pickup",
    "jfk_dropoff",
    "lga_pickup",
    "lga_dropoff",
    "is_many_passengers",
    "store_and_fwd_flag",
]
numeric_features = ["log_haversine", "manhattan_dist"]

features = ColumnTransformer(
    [
        ("scaling", StandardScaler(), numeric_features),
        (
            "ohe_all",
            OneHotEncoder(
                handle_unknown="ignore", drop="if_binary", sparse_output=False
            ),
            categorical,
        ),
    ]
)

pipeline_no_regressor = Pipeline(
    [
        (
            "features",
            FeatureUnion([("pickup_dropoff", pickup_dropoff), ("features", features)]),
        ),
        # ("regression", Ridge()),
    ]
)

# pickup_dropoff.fit_transform(xval.collect()).select(pl.sum('occ_ohe__occurrence_rank_1', 'occ_ohe__occurrence_rank_99'))

In [435]:
xtrain_trsf = pipeline_no_regressor.fit_transform(xtrain.collect()).lazy()
xval_trsf = pipeline_no_regressor.fit_transform(xval.collect()).lazy()

Заново обучите модель (`Ridge`, если она дала более высокое качество в предыдущих экспериментах, и `Lasso` иначе) на новых даннных и посчитайте качество на тестовой выборке

In [ ]:
model5 = Ridge()
model5.fit(xtrain_trsf.collect(), xtrain.select("log_trip_duration").collect()["log_trip_duration"])

In [ ]:
ytrain_pred = model5.predict(xtrain_trsf.collect())
yval_pred = model5.predict(xval_trsf.collect())

print(
    f"""
    Constant prediction RMSLE {rmsle_best_const:.3f}
    Train RMSLE: {mean_squared_error(xtrain.select('log_trip_duration').collect()['log_trip_duration'], ytrain_pred)**.5:.3f}
    Validation RMSLE: {mean_squared_error(xval.select('log_trip_duration').collect()['log_trip_duration'], yval_pred)**.5:.3f}
"""
)

In [ ]:
np.exp(
    xtrain.select("log_trip_duration").mean().collect().item()
    + np.array([0.796, 0.534]).reshape(-1, 1) * np.linspace(-2, 2, 5)
)

**Задание 17 (бонус, 2 балла)**.

Разумеется, погружаться в feature engineering можно ещё очень долго. Ваша задача - придумать какие-то новые признаки, которые сделают модель ещё лучше!! За улучшение функционала ошибки на каждые 0.005 на тестовой выборке относительно `best_rmsle` будет даваться 0.5 бонусных балла. Всего за этот пункт можно получить до 2 бонусных баллов.

При построении признаков старайтесь не допустить утечки целевой переменной (подробнее про это можно почитать в материалах 1-го семинара) — в противном случае хорошего качества на тестовой выборке достичь не получится.

Какие могут быть идеи для вдохновения:

1. Трансформер, который строит разбиение карты по шестигранной решётке с помощью библиотеки [H3](https://github.com/uber/h3-py) и вычисляет признаки на основе такого разбиения, по аналогии с квадратной сеткой  
> Важно: производительность библиотеки существенно зависит от количества шестиугольников на карте (определяется параметром resolution). Подберите такое разрешение, при котором ваш код будет работать за приемлемое время.    

2. Мы пока что никак не использовали историчность и взаимосвязь в данных, хотя информация о том, что было ранее в эти/похожие дни тоже может пригодиться. Попробуйте поискать соседей и/или построить агрегации, например, при помощи groupby, посчитайте какие-либо статистики в пределах какого-то окна (подберите всё это сами, постарайтесь, чтобы это было что-то разумное), и присоедините к основному датафрейму
> Важно: учтите, что разных вариантов агрегаций и статистик существует очень много. Не стесняйтесь выкидывать фичи, которые вам не нравятся (помните про свойство Lasso), используйте эффективные способы хранения данных ([wink](https://www.kaggle.com/code/demche/polars-memory-usage-optimization)-[wink](https://docs.scipy.org/doc/scipy/reference/sparse.html))

3. В конце концов можно использовать альтернативные способы преобразования данных. Например, использовать другой scaler, другую кодировку категориальных фичей, другие гиперпараметры и пр. Но помните, что на данный момент мы ограничены только `Ridge` и `Lasso`

In [ ]:
# ╰( ͡° ͜ʖ ͡° )つ──☆*:・ﾟ

Вставьте картинку, описывающую ваш опыт выполнения этого ДЗ.